In [20]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import visual_tools as vt
from utils import gt_and_modeling_dfs as prepare_df

print(sys.executable)
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/kartikarya/Documents/Code/SMProject/env_mupppet/bin/python


# Preprocessing Episodes

> Train-test split

> **Choice of split:**\
> There are 4 main characters that we need to identify so the split time from a given episode is selected based on the equal (rough idea) no of apperances of the all all character in both splits.

In [ ]:
# fps -> acc to processign requirements
FPS_TO_SAVE = 25

EPISODES = {
    "Muppets-02-01-01": {
        "path": "../data/raw/video/Muppets-02-01-01.avi",
        "intro_skip_timestamp": "00:17",
        "train_split_timestamp": "19:30",
        "ground_truth_path": "../data/muppets-gt-2025wt/Ground_Truth_New_01.xlsx"
    },
    "Muppets-02-04-04": {
        "path": "../data/raw/video/Muppets-02-04-04.avi",
        "intro_skip_timestamp": "00:15",
        "train_split_timestamp": "19:52",
        "ground_truth_path": "../data/muppets-gt-2025wt/Ground_Truth_New_04.xlsx"
    },
    "Muppets-03-04-03": {
        "path": "../data/raw/video/Muppets-03-04-03.avi",
        "intro_skip_timestamp": "00:09",
        "train_split_timestamp": "19:54",
        "ground_truth_path": "../data/muppets-gt-2025wt/Ground_Truth_New_03.xlsx"
    }
}


EPISODE_GT = {
    "Muppets-02-01-01": "../data/muppets-gt-2025wt/Ground_Truth_New_01.xlsx",
    "Muppets-03-04-03": "../data/muppets-gt-2025wt/Ground_Truth_New_03.xlsx",
    "Muppets-02-04-04": "../data/muppets-gt-2025wt/Ground_Truth_New_04.xlsx",
}

In [ ]:
for episode_name, ep in EPISODES.items():

    print(f"\n=== Processing {episode_name} ===")

    video_path = ep["path"]
    intro_ts   = ep["intro_skip_timestamp"]
    split_ts   = ep["train_split_timestamp"]

    # -----------------------------
    # 1. Video info
    # -----------------------------
    info = vt.video_info(video_path)
    print(info)

    # -----------------------------
    # 2. Extract frames
    # -----------------------------
    frames_dir, total_saved = vt.extract_frames(
        video_path,
        fps_to_save=FPS_TO_SAVE
    )

    # -----------------------------
    # 3. Load metadata + skip intro
    # -----------------------------
    frames, _, _ = vt.extract_frames_meta(
        video_path,
        intro_timestamp=intro_ts,
        fps_to_save=FPS_TO_SAVE
    )

    # -----------------------------
    # 4. Split train / test
    # -----------------------------
    train_frames, test_frames = vt.split_frames(
        frames,
        video_path,
        split_timestamp=split_ts,
        fps_to_save=FPS_TO_SAVE
    )

    print(
        f"[DONE] {episode_name} | "
        f"frames={total_saved}, "
        f"train={len(train_frames)}, "
        f"test={len(test_frames)}"
    )


=== Processing Muppets-02-01-01 ===
{'fps': 25.0, 'width': 720, 'height': 544, 'num_frames': 38682, 'fourcc': 877677894, 'duration_sec': 1547.28}
[frames ok] saved=38682 frames in ../data/raw/video/Muppets-02-01-01-frames
[YoHoo! split done :>] saved frames: train=29250, test=9007
[DONE] Muppets-02-01-01 | frames=38682, train=29250, test=9007

=== Processing Muppets-02-04-04 ===
{'fps': 25.0, 'width': 720, 'height': 544, 'num_frames': 38707, 'fourcc': 877677894, 'duration_sec': 1548.28}
[frames ok] saved=38707 frames in ../data/raw/video/Muppets-02-04-04-frames
[YoHoo! split done :>] saved frames: train=29800, test=8532
[DONE] Muppets-02-04-04 | frames=38707, train=29800, test=8532

=== Processing Muppets-03-04-03 ===
{'fps': 25.0, 'width': 704, 'height': 544, 'num_frames': 38499, 'fourcc': 877677894, 'duration_sec': 1539.96}
[frames ok] saved=38499 frames in ../data/raw/video/Muppets-03-04-03-frames
[YoHoo! split done :>] saved frames: train=29850, test=8424
[DONE] Muppets-03-04-03 |

In [28]:
for episode_name, gt_path in EPISODE_GT.items():
    frames_dir = f"../data/raw/video/{episode_name}-frames"
    vt.sanity_check_frames_vs_gt(frames_dir, gt_path)

[WARNING] Frame count mismatch!
Extracted frames: 38682, GT rows: 38680
[WARNING] Frame count mismatch!
Extracted frames: 38499, GT rows: 38497
[WARNING] Frame count mismatch!
Extracted frames: 38707, GT rows: 38705


### Merging gt for all episodes

In [ ]:
all_ep_gt_df = prepare_df.all_ep_gt()

display(all_ep_gt_df.head())
display(all_ep_gt_df.tail())
print(all_ep_gt_df["Video"].value_counts())

Consolidated GT: 115885 rows, 10 columns


,Video,Frame_number,Timestamp,Kermit,Pigs,Miss Piggy,Cook,StatlerWaldorf,Rowlf the Dog,Fozzie Bear
0,211,0,00:00:00,0,0,0,0,0,0,0
1,211,1,00:00:00.040000,0,0,0,0,0,0,0
2,211,2,00:00:00.080000,0,0,0,0,0,0,0
3,211,3,00:00:00.120000,0,0,0,0,0,0,0
4,211,4,00:00:00.160000,0,0,0,0,0,0,0


,Video,Frame_number,Timestamp,Kermit,Pigs,Miss Piggy,Cook,StatlerWaldorf,Rowlf the Dog,Fozzie Bear
115880,244,38701,00:25:48.040000,0,0,0,0,0,0,0
115881,244,38702,00:25:48.080000,0,0,0,0,0,0,0
115882,244,38703,00:25:48.120000,0,0,0,0,0,0,0
115883,244,38704,00:25:48.160000,0,0,0,0,0,0,0
115884,244,38705,00:25:48.200000,0,0,0,0,0,0,0


Video
244    38706
211    38681
343    38498
Name: count, dtype: int64


# Feature Extraction

### Getting train and test features features

In [ ]:
episode_name = "Muppets-02-01-01"
features_dir = f"../data/processed/video/{episode_name}/features/"

train_frames_small = ["frame175.jpg", "frame176.jpg", "frame177.jpg", "frame311.jpg", "frame312.jpg", "frame313.jpg", "frame1648.jpg", "frame1692.jpg", "frame4678.jpg"]

test_frames_small = ["frame5281.jpg", "frame5382.jpg", "frame5380.jpg", "frame5434.jpg", "frame5427.jpg"]

train_frames_dir = f"../data/processed/video/{episode_name}/train/"
test_frames_dir  = f"../data/processed/video/{episode_name}/test/"

# --- Dominant color ---
train_colors, train_colors_csv = vt.dominant_color(train_frames_dir, episode_name, frame_files=train_frames_small)
test_colors,  test_colors_csv  = vt.dominant_color(test_frames_dir, episode_name, frame_files=test_frames_small)

# --- Green mask feature ---
train_green, train_green_csv = vt.green_mask(train_frames_dir, episode_name, frame_files=train_frames_small)
test_green,  test_green_csv  = vt.green_mask(test_frames_dir, episode_name, frame_files=test_frames_small)

# --- Edge feature ---
train_edges, train_edges_csv = vt.detect_edges(train_frames_dir, episode_name, frame_files=train_frames_small)
test_edges,  test_edges_csv  = vt.detect_edges(test_frames_dir, episode_name, frame_files=test_frames_small)


[Dominant color] saved 9 frames -> ../data/processed/video/Muppets-02-01-01/features/dominant_color_train.csv
[Dominant color] saved 5 frames -> ../data/processed/video/Muppets-02-01-01/features/dominant_color_test.csv
[Green mask] saved 9 frames -> ../data/processed/video/Muppets-02-01-01/features/green_mask_train.csv
[Green mask] saved 5 frames -> ../data/processed/video/Muppets-02-01-01/features/green_mask_test.csv
[Edge magnitude] saved 9 frames -> ../data/processed/video/Muppets-02-01-01/features/edge_magnitude_train.csv
[Edge magnitude] saved 5 frames -> ../data/processed/video/Muppets-02-01-01/features/edge_magnitude_test.csv


### Making dataframes for model processing

In [ ]:
# combine train features
X_train = pd.concat([
    pd.DataFrame(train_colors).drop(columns='frame'),
    pd.DataFrame(train_edges).drop(columns='frame'),
    pd.DataFrame(train_green).drop(columns='frame')
], axis=1)

# combine test features
X_test = pd.concat([
    pd.DataFrame(test_colors).drop(columns='frame'),
    pd.DataFrame(test_edges).drop(columns='frame'),
    pd.DataFrame(test_green).drop(columns='frame')
], axis=1)

## Model training